In [ ]:
fold = 0

In [ ]:
import pandas as pd

sentence_only_with_folds_df = pd.read_csv("ANON")
print(sentence_only_with_folds_df.test_fold.unique())
sentence_only_with_folds_df.head()

In [ ]:
print(len(sentence_only_with_folds_df))
train_df = sentence_only_with_folds_df[sentence_only_with_folds_df.test_fold != fold]
# remove columns text-id, id, test_fold
train_df = train_df.drop(columns=['id', 'test_fold'])
print(len(train_df))
train_df.head()

In [ ]:
# shuffle train_df with seed 42
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df.head()

In [ ]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('bert', 'bert-base-german-cased', num_labels=max(train_df.label) + 1, use_cuda=False, args={'output_dir': f'output-fold-{fold}', 'num_train_epochs': 3})
model.train_model(train_df)

In [ ]:
test_df = sentence_only_with_folds_df[sentence_only_with_folds_df.test_fold == fold]

# find F1 score per label
from sklearn.metrics import f1_score, balanced_accuracy_score
import numpy as np

def f1_score_per_label(y_true, y_pred):
    return np.array([f1_score(y_true == i, y_pred == i) for i in range(max(y_true) + 1)])

predictions, raw_outputs = model.predict(test_df.text.to_list())
list_of_f1 = list(f1_score_per_label(test_df.label, predictions))

overall_f1 = f1_score(test_df.label, predictions, average='weighted')
overall_balanced_accuracy = balanced_accuracy_score(test_df.label, predictions)
with open(f'f1-fold-{fold}.txt', 'w') as f:
    for item in list_of_f1 + [overall_f1, overall_balanced_accuracy]:
        f.write("%s\n" % item)

In [ ]:
print(list_of_f1)